## Author 
****
Karen Rugerio Armenta

## Business case
****
Multivariate data preprocessing and time series processing

## Case Description
****
Make an exploratory analysis of the variables using the variance-covariance matrix, and correlation matrix.

Run statistical tests to detect outliers and leverage points.

Make a multicollinearity analysis and  its implications in the model.

Estimate coefficients and standard errors of the regression model and interpret the model solving the problems presented along the way.



#Importing Libraries

In [1]:
#Libraries
import warnings
warnings.simplefilter(action='ignore')
import pandas as pd              
import matplotlib.pyplot as plt   
import numpy as np                
from sklearn import linear_model
from scipy.stats.mstats import winsorize
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
from sklearn.metrics import r2_score, accuracy_score
from pyparsing import alphas
from pandas.core.api import DataFrame
import statsmodels.api as sm
from numpy.linalg import matrix_power

#Creating DataFrame focusing in Finance and Insurance industires

In [2]:
#Extraction
from google.colab import drive
drive.mount("/content/gdrive")  
%cd "/content/gdrive/MyDrive/Colab Notebooks/Bloque1/Estadística - M1/final"
np.set_printoptions(suppress=True)

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks/Bloque1/Estadística - M1/final


In [3]:
#DataFrame creation
dfusfirms2022 = pd.read_csv('usfirms2022.csv')
dfusfirms2022 = dfusfirms2022[['Ticker', 'Name', 'N', 'Class','Type of Asset', 'Sector\nEconomatica', 'Sector NAICS\nlevel 1']]
dfus2022q2a = pd.read_csv('us2022q2a.csv')
dfus2022q2a = dfus2022q2a[['firm', 'q','revenue','cogs','sgae', 'otheropexp','extraincome','finexp','incometax','totalassets', 'totalliabilities', 'shortdebt', 'longdebt', 'stockholderequity', 'adjprice', 'originalprice', 'sharesoutstanding', 'fiscalmonth', 'year','cto']]
df = pd.merge(dfus2022q2a,dfusfirms2022, left_on='firm', right_on='Ticker')
df = df.drop(columns='Ticker')
#Choose only Financial Data. I decided to filter the financial data by the Finance and Insurance name in Sector Economatica
df = pd.DataFrame(df.loc[df['Sector\nEconomatica'] == 'Finance and Insurance'])

In [4]:
#Verify the number of firms content in the Finance and Insurance dataset
firms = df['firm'].drop_duplicates()
firms

360       AAME
1170      ABCB
2064      ABTX
2244        AC
3594      ACNB
          ... 
316346    WTBA
316436    WTFC
316616     WTM
320486       Y
322466    ZION
Name: firm, Length: 515, dtype: object

In [5]:
df['qdate'] = pd.PeriodIndex(df['q'], freq="Q")
df.set_index(['firm','qdate'], inplace=True)
df

q   revenue     cogs  sgae  otheropexp  extraincome  finexp  \
firm qdate                                                                      
AAME 2000Q1  2000q1       NaN      NaN   NaN         NaN          NaN     NaN   
     2000Q2  2000q2       NaN      NaN   NaN         NaN          NaN     NaN   
     2000Q3  2000q3       NaN      NaN   NaN         NaN          NaN     NaN   
     2000Q4  2000q4       NaN      NaN   NaN         NaN          NaN     NaN   
     2001Q1  2001q1       NaN      NaN   NaN         NaN          NaN     NaN   
...             ...       ...      ...   ...         ...          ...     ...   
ZION 2021Q2  2021q2  570000.0  15000.0   0.0         0.0    -100000.0     0.0   
     2021Q3  2021q3  569000.0  14000.0   0.0         0.0    -244000.0     0.0   
     2021Q4  2021q4  566000.0  13000.0   0.0         0.0    -284000.0     0.0   
     2022Q1  2022q1  555000.0  11000.0   0.0         0.0    -289000.0     0.0   
     2022Q2  2022q2  608000.0  15000.0   0.0         0.0    -333000.0     0.0   

             incometax  totalassets  totalliabilities  ...  sharesoutstanding  \
firm qdate                                             ...                      
AAME 2000Q1        NaN          NaN               NaN  ...                NaN   
     2000Q2        NaN          NaN               NaN  ...                NaN   
     2000Q3        NaN          NaN               NaN  ...                NaN   
     2000Q4        NaN          NaN               NaN  ...                NaN   
     2001Q1        NaN          NaN               NaN  ...                NaN   
...                ...          ...               ...  ...                ...   
ZION 2021Q2   101000.0   87208000.0        79175000.0  ...         163815.613   
     2021Q3    71000.0   88306000.0        80532000.0  ...         162070.463   
     2021Q4    56000.0   93200000.0        85737000.0  ...         156463.463   
     2022Q1    52000.0   91126000.0        84832000.0  ...         151574.325   
     2022Q2    57000.0   87784000.0        82152000.0  ...         151358.748   

             fiscalmonth  year  cto  \
firm qdate                            
AAME 2000Q1          NaN  2000    1   
     2000Q2          NaN  2000    2   
     2000Q3          NaN  2000    3   
     2000Q4          NaN  2000    4   
     2001Q1          NaN  2001    1   
...                  ...   ...  ...   
ZION 2021Q2          6.0  2021    2   
     2021Q3          9.0  2021    3   
     2021Q4         12.0  2021    4   
     2022Q1          3.0  2022    1   
     2022Q2          6.0  2022    2   

                                                   Name     N  Class  \
firm qdate                                                             
AAME 2000Q1                      Atlantic American Corp   353    Com   
     2000Q2                      Atlantic American Corp   353    Com   
     2000Q3                      Atlantic American Corp   353    Com   
     2000Q4                      Atlantic American Corp   353    Com   
     2001Q1                      Atlantic American Corp   353    Com   
...                                                 ...   ...    ...   
ZION 2021Q2  Zions Bancorporation, National Association  3596    Com   
     2021Q3  Zions Bancorporation, National Association  3596    Com   
     2021Q4  Zions Bancorporation, National Association  3596    Com   
     2022Q1  Zions Bancorporation, National Association  3596    Com   
     2022Q2  Zions Bancorporation, National Association  3596    Com   

             Type of Asset    Sector\nEconomatica  Sector NAICS\nlevel 1  
firm qdate                                                                
AAME 2000Q1          Stock  Finance and Insurance  Finance and Insurance  
     2000Q2          Stock  Finance and Insurance  Finance and Insurance  
     2000Q3          Stock  Finance and Insurance  Finance and Insurance  
     2000Q4          Stock  Finance and Insurance  Finance and Insurance  
     2001Q1    

In [6]:
#Calculating operating profit margin
df['ebit'] = df.revenue - df.cogs - df.sgae - df.otheropexp
df.loc[df['revenue'] < 0, 'operatingMargin'] = df.ebit / (df.revenue)  #Avoid dividing by 0
df.loc[df['revenue'] > 0, 'operatingMargin'] = df.ebit / (df.revenue)  #Avoid dividing by 0
df.loc[df['revenue'] == 0, 'operatingMargin'] = 'NaN'
#df.dropna(subset=['operatingMargin'])

In [7]:
#Getting the book to market ratio

#Fist we get the book value
df['bookValue'] = df.totalassets - df.totalliabilities 
#Then we get the market value
df['marketValue'] = df.originalprice * df.sharesoutstanding 

#Then we calculate the book to market ratio by dividing book value / market value
df.loc[df['marketValue'] < 0, 'bookToMarketRatio'] = df.bookValue / df.marketValue  #Avoid dividing by 0
df.loc[df['marketValue'] > 0, 'bookToMarketRatio'] = df.bookValue / df.marketValue  #Avoid dividing by 0
df.loc[df['marketValue'] == 0, 'bookToMarketRatio'] = 'NaN'
#df.dropna(subset=['bookToMarketRatio'])

In [8]:
#Getting short financial leverage
df['shortFinancialLeverage'] = df.shortdebt / df.totalassets
df.loc[df['totalassets'] < 0, 'shortFinancialLeverage'] = df.shortdebt / df.totalassets #Avoid dividing by 0
df.loc[df['totalassets'] > 0, 'shortFinancialLeverage'] = df.shortdebt / df.totalassets  #Avoid dividing by 0
df.loc[df['totalassets'] == 0, 'shortFinancialLeverage'] = 'NaN'


In [9]:
#Getting long financial leverage
df.loc[df['totalassets'] < 0, 'longFinancialLeverage'] = df.longdebt / df.totalassets  #Avoid dividing by 0
df.loc[df['totalassets'] > 0, 'longFinancialLeverage'] = df.longdebt / df.totalassets  #Avoid dividing by 0
df.loc[df['totalassets'] == 0, 'longFinancialLeverage'] = 'NaN'


In [10]:
#Getting EPS metrics
df['netincome'] = df.revenue - df.cogs - df.sgae - df.otheropexp - df.incometax - df.finexp + df.extraincome 
df['EPS'] = df.netincome / df.sharesoutstanding
#Getting EPSP metrics
df.loc[df['originalprice'] < 0, 'EPSP'] = df.EPS / df.originalprice #Avoid dividing by 0
df.loc[df['originalprice'] > 0, 'EPSP'] = df.EPS / df.originalprice  #Avoid dividing by 0
df.loc[df['originalprice'] == 0, 'EPSP'] = 'NaN'

In [11]:
#Calculating the Continuosly Compunded Return
df['r'] = np.log(df['adjprice'] - np.log(df['adjprice'].shift(4)))

In [12]:
#As we have the conditions to avoid dividing by 0 because we do not want to have infinite values, we obtain
#objects, so we have to convert this objects into floats so we can work with them
df = df.astype({'operatingMargin':'float64', 'bookToMarketRatio':'float64', 'shortFinancialLeverage':'float64', 'longFinancialLeverage':'float64','EPSP':'float64'})

In [13]:
#Getting penultimate quarter of the year (2022Q1)
df = df.groupby('firm').tail(2) 
df = df.groupby('firm').head(1)
df = df[['operatingMargin','bookToMarketRatio','shortFinancialLeverage','longFinancialLeverage','EPSP', 'r',]]

As we can see, the Short Financial Leverage has a big number of NaN values, therefore we will not use it for the following calcultaions

In [14]:
df = df.drop(columns='shortFinancialLeverage').reset_index().dropna()

#**1 - Exploratory analysis of the variables::**

##**A - Calculates variance-covariance matrix, as well as correlation matrix of the independent and dependent variables. Explain what variance, covariance, and correlation are. Interpret the correlation matrix. You have to use matrix algebra and check results with Python functions.**

###**Variance-Covariance matrix:**

**Variance is a statistical measurement of the spread between numbers in a data set.This measurement is used when quantifying how spread out the values are in a data set is required. The larger the variance, the more spread out the values are. The variance is always greater than or equal to 0.**

**In finance and accounting, the variance is used to show the average of the squared distances of the individual returns in a portfolio from their mean value.** 

**Covariance is used to measure how two variables change when compared to each other.**

**The covariance can be less than 0, equal to 0, or greater than 0:**

- **1.- When the covariance is less than 0: in this case, there is a negative relationship, so that X and Y are two variables that are inversely proportional to each other; i.e, when variable Y increases, variable X decreases.**
- **2.- When the covariance is greater than 0: in this case, there is a positive relationship, so that X and Y are two variables directly proportional to each other; i.e, when variable X increases, variable Y it does too.**
- **3.- When the covariance acquires a value equal to 0: in this case, the relationship between one variable and another variable is non-existent, which means that the covariance will be equal to 0 regardless of whether any of the two variables increases or decreases.**

**In a financial context, covariance refers to the returns of two different investments over time when compared to different variables, such as stocks or other marketable securities.**

**On the other hand, the variance-covariance matrix is a square matrix of dimension n*m that groups the variances in the main diagonal and the covariances in the elements outside the main diagonal. The formula to calculate the variance-covariance matrix is as follows:**

**(1/n-1) [X' X - 1/N (X'1) (X'1)' ]**


**This formula was implemented in Python with the help of the dot functions (to calculate the dot product) and the transpose function (to calculate the corresponding transposed matrix) from numpy.**

In [15]:
#Define variables we are going to use, stroring them in the X dataframe
X = df[['operatingMargin','bookToMarketRatio','longFinancialLeverage','EPSP', 'r']]
Xt = X.transpose()

# Obtain how many rows(N) and columns(M) we have
N, M = X.shape
# Create a ones matrix so I can do the (X'1) operations
ones = np.ones((N,1))
XtOnes = Xt.dot(ones)
#Doing the operation that help us obtain the variance-covariance matrix:
# VarCov(X) = (1/n-1)[X' X - 1/N (X'1) (X'1)' ]
varCov = (1/(N-1))*(Xt.dot(X)-(1/N)*(XtOnes).dot(XtOnes.transpose()))
varCov

,operatingMargin,bookToMarketRatio,longFinancialLeverage,EPSP,r
operatingMargin,3.338279,0.179897,-0.019911,0.001116,0.088395
bookToMarketRatio,0.179897,0.362198,-0.008694,0.010531,-0.255905
longFinancialLeverage,-0.019911,-0.008694,0.022832,0.000138,0.006470
EPSP,0.001116,0.010531,0.000138,0.003496,0.003977
r,0.088395,-0.255905,0.006470,0.003977,1.214980


**We can prove our results with the pandas function .cov()**

In [16]:
#Python function that help us get the correlation
df.cov()

,operatingMargin,bookToMarketRatio,longFinancialLeverage,EPSP,r
operatingMargin,3.338279,0.179897,-0.019911,0.001116,0.088395
bookToMarketRatio,0.179897,0.362198,-0.008694,0.010531,-0.255905
longFinancialLeverage,-0.019911,-0.008694,0.022832,0.000138,0.006470
EPSP,0.001116,0.010531,0.000138,0.003496,0.003977
r,0.088395,-0.255905,0.006470,0.003977,1.214980


**According to the previously calculated variance-covariance matrix, the following can be inferred (Based on the result of the main diagonal):**

 - **The variance of “operatingMargin” is ≈ 3.33**
 - **The variance of “bookToMarketRatio” is ≈ 0.36**
 - **The variance of “longFinancialLeverage” is ≈ 0.022**
 - **The variance of “EPSP” is ≈ 0.0034**
 - **The variance of “r” is ≈ 1.21**

**The covariance between variables can be obtained from the result obtained outside the main diagonal. For example, the covariance between operatingMargin and bookToMarketRatio is ≈ 0.17.**

**Finally, the result obtained by the formula implementation and the result of the variance-covariance matrix obtained using the method .cov() was compared having the exact same value.** 


###**Correlation Matrix**

**A correlation matrix is a table that help us to have a better understanding of our variables and their correlation coefficients with other variables and to identify patterns in the given data.** 

**Correlation coefficient values are between +/- 1.0. A really strong correlation can be considered with an absolute value over 0.6, and when doing the correlation matrix we can see 1s in the diagonal because of course one viarbe is completely correlated with its own**

In [17]:
#We obtain the varaince with the diagonal 
N, M = df[['operatingMargin','bookToMarketRatio','longFinancialLeverage','EPSP', 'r']].shape
variance = np.diag(varCov).reshape(1, M)
#Calculate the standar deviation
std = np.sqrt(variance) * np.sqrt(variance.transpose()) 
#Get the correlation matrix
correlationMatrix = varCov / std
correlationMatrix

,operatingMargin,bookToMarketRatio,longFinancialLeverage,EPSP,r
operatingMargin,1.000000,0.163603,-0.072122,0.010334,0.043892
bookToMarketRatio,0.163603,1.000000,-0.095601,0.295916,-0.385765
longFinancialLeverage,-0.072122,-0.095601,1.000000,0.015401,0.038844
EPSP,0.010334,0.295916,0.015401,1.000000,0.061017
r,0.043892,-0.385765,0.038844,0.061017,1.000000


In [18]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,operatingMargin,bookToMarketRatio,longFinancialLeverage,EPSP,r
operatingMargin,1.000000,0.163603,-0.072122,0.010334,0.043892
bookToMarketRatio,0.163603,1.000000,-0.095601,0.295916,-0.385765
longFinancialLeverage,-0.072122,-0.095601,1.000000,0.015401,0.038844
EPSP,0.010334,0.295916,0.015401,1.000000,0.061017
r,0.043892,-0.385765,0.038844,0.061017,1.000000


##**B - Run statistical tests to detect outliers and leverage points. You have to use matrix algebra for the tests and clearly explain how the tests work. You can use Python functions to check results.**

###Leverage points detection

**A leverage point is an observation that has an unusual predictor value that is very different from the the observations. If this leverage point is an influence point means that its removal from the data set would cause a large change in the estimated reggression model coefficients. Removing the high leverage points usually makes the prediction more accurate**

**To be considered as an influence point it has to have extreme values. To help us identify them it exists a common rule to flag any observation whose leverage value, is more than 3 times larger than the mean leverage value:**

- **3((k+1)/n)**

**where:**
 - **k = number of x variables**
 - **n = number of values**

**If our point is bigget than 3((k+1)/n) then we flag the observations as "Unusual X" because potentially is a large influence. What we want is not to have high leverage poins, so we can have better predictions**

In [19]:
#Define independant variables
X = df[['operatingMargin','bookToMarketRatio','longFinancialLeverage','EPSP']]
Xt = X.transpose()
#Define dependant varible
y = df['r']
y = y.to_numpy().reshape(476,1) #Because it has to be of shape nx1
# Obtain how many rows(N) and columns(M) we have
N, M = X.shape
# Create a ones matrix so I can do the (X'1) operations
ones = np.ones((N,1))
XtOnes = Xt.dot(ones)
#Doing the operation that help us obtain the ŷ:
#Formula ŷ = Hy
#Formula H =X(X′X)−1X′
H =  X.dot((matrix_power(Xt.dot(X),-1)).dot(Xt))
yHat = H.dot(y)
lpdf = pd.DataFrame(yHat) #Leverage Points Data Frame
lpdf.columns = ['yHat']
lpdf['y'] = y
N, M = lpdf.shape
#Obtainig high leverage points with the 3(k+1/n) formula
#k are the number of parameters considered
highLeveragePoints = 3 * (4+1/N)
#prints only the ones that are over the high leverage formula
lpdf[lpdf['yHat'].abs()>highLeveragePoints]

,yHat,y
113,13.898289,2.526965
209,13.015073,2.037274
213,16.077826,0.947803


**I decided to run a Multivariate Linear Regression so we can see the improvement in the model when dropping the leverage points. First we see the Multivariate Linear Regression before droping the leverage points**

In [20]:
#Multivariate Linear Regression with leverage points
#First we have to drop the Infinite values and Nas
df.replace([np.inf, -np.inf], np.nan, inplace=True)
dfML = df.reset_index().drop(columns=['firm','qdate']).dropna()
#define dependant variable (response)
y = dfML['r']
#define independant variables (predictor)
x = dfML[['operatingMargin','bookToMarketRatio','longFinancialLeverage','EPSP']]
#add constant to predictor variables
x = sm.add_constant(x)
#fit linear regression model
model = sm.OLS(y, x).fit()
#view model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      r   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     28.69
Date:                Fri, 14 Oct 2022   Prob (F-statistic):           2.37e-21
Time:                        21:04:30   Log-Likelihood:                -669.37
No. Observations:                 476   AIC:                             1349.
Df Residuals:                     471   BIC:                             1370.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     4.03

**Dropping the leverage poins**

In [21]:
#Droping rows that are leverage points on the main dataframe
df = df.drop(labels=[113,209,213], axis=0)

**Multivariate Linear Regression without leverage points**

In [22]:
#Multivariate Linear Regression 
#First we have to drop the Infinite values and Nas
df.replace([np.inf, -np.inf], np.nan, inplace=True)
dfML = df.reset_index().drop(columns=['firm','qdate']).dropna()
#define dependant variable (response)
y = dfML['r']
#define independant variables (predictor)
x = dfML[['operatingMargin','bookToMarketRatio','longFinancialLeverage','EPSP']]
#add constant to predictor variables
x = sm.add_constant(x)
#fit linear regression model
model = sm.OLS(y, x).fit()
#view model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      r   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     35.80
Date:                Fri, 14 Oct 2022   Prob (F-statistic):           4.17e-26
Time:                        21:04:30   Log-Likelihood:                -651.28
No. Observations:                 473   AIC:                             1313.
Df Residuals:                     468   BIC:                             1333.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     4.27

**As we can see, there are only 3 values that meet this condition, so we have to take them out of the dataset. Running the linear regression with the outliers it is an R-aquared of  0.196 while dropping the leverahe points it has an R-squared value of 0.234.Which means there was an improvement of around 4% in the R-squared**

**The way I used to corroborate this result is applying the 3(k+1/n) formula and getting the poits out of the dataset. If the model gets better means it was a good job done**

###Outliers points detection

**Jim Frost from Statistics Intuitive defines an outlier as an extremely high or extremely low data point relative to the nearest data point and the rest of the neighboring co-existing values in a data graph or dataset you're working with.**

**Therefore we can interpret an outlier as a point that do not follow the general trend of the rest of the data, therefore if it is an ifluential point it can affect our linear regression results.**

In [23]:
#Define independant variables
X = df[['operatingMargin','bookToMarketRatio','longFinancialLeverage','EPSP']]
Xt = X.transpose()
#Define dependant varible
Y = df['r']
# Obtain how many rows(N) and columns(M) we have
N, M = X.shape
y = Y.to_numpy().reshape(N,1) #Because it has to be of shape nx1
# Create a ones matrix so I can do the (X'1) operations
ones = np.ones((N,1))
XtOnes = Xt.dot(ones)
#Doing the operation that help us obtain the ŷ:
#Formula ŷ = Hy
#To calculate H we use the followingv formula:
#X(X′X)−1X′
H =  X.dot((matrix_power(Xt.dot(X),-1)).dot(Xt))
yHat = H.dot(y)
odf = pd.DataFrame(yHat) #Outliers Data Frame
odf.columns = ['yHat']
odf['y'] = Y
#Calculate ordinary residual errors (ei)
#ei= yi − ŷi
odf['h'] = np.diag(H)
odf['ei'] = odf.y - odf.yHat
#Getting the mean squared error used in the std residual error formula
mse = mean_squared_error(odf['y'], odf['yHat'])
#Apply the Standardized residuals formula (ri)
# ri = ei / sqrt(mse(1-hii)) 
odf['stdresidualError'] = odf.ei / np.square(mse*(1-odf.h))
#if a standardized residual is larger than 3 (in absolute value) is considered as an outlier
odf[odf['stdresidualError'].abs()>3]

,yHat,y,h,ei,stdresidualError
174,-6.81767,0.168588,0.843386,6.986258,15.216687


**By this time we have a R-squared of 0.234 after dropping the leverage poins, now we will drop the outlier found and see how much improve our model**

In [24]:
#Droping rows that are leverage points on the main dataframe
df = df.drop(labels=[174], axis=0)

**Multivariate Linear Regression without outliers**

In [25]:
#Multivariate Linear Regression 
#First we have to drop the Infinite values and Nas
df.replace([np.inf, -np.inf], np.nan, inplace=True)
dfML = df.reset_index().drop(columns=['firm','qdate']).dropna()
#define dependant variable (response)
y = dfML['r']
#define independant variables (predictor)
x = dfML[['operatingMargin','bookToMarketRatio','longFinancialLeverage','EPSP']]
#add constant to predictor variables
x = sm.add_constant(x)
#fit linear regression model
model = sm.OLS(y, x).fit()
#view model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      r   R-squared:                       0.238
Model:                            OLS   Adj. R-squared:                  0.231
Method:                 Least Squares   F-statistic:                     36.45
Date:                Fri, 14 Oct 2022   Prob (F-statistic):           1.59e-26
Time:                        21:04:30   Log-Likelihood:                -644.70
No. Observations:                 472   AIC:                             1299.
Df Residuals:                     467   BIC:                             1320.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     4.47

**As we can see, it is not a realy big improvement, but it did improved from 0.234 to 0.238 so in this case the outlier seems not to be influential. By deleting this point, the mean square error is not substantially deinflated, as this help us increase our confidence and prediction intervals just a few. However the predicted responses are not affected by the exclusion of the outlier.**

#**2 - Multicollinearity analysis explaining the test and implications in the model.**

**To calculate the multicollinearity I used a VIF function that will help us know the variance inflation factor, in statistics is used to know the severity of multicollinearity in the ordinary least square (OLS) regression analysis. Multicollinearity inflates the variance and type II error. It makes the coefficient of a variable consistent but unreliable**

In [26]:
def calculate_vif(df, features):    
    vif, tolerance = {}, {}    #features
    for feature in features:
        # extract all the other features to regress against
        X = [f for f in features if f != feature]        
        X, y = df[X], df[feature]        # extract r-squared 
        r2 = LinearRegression().fit(X, y).score(X, y)                
        
        # calculate tolerance
        tolerance[feature] = 1 - r2        # calculate VIF
        vif[feature] = 1/(tolerance[feature])    # return VIF DataFrame
    return pd.DataFrame({'VIF': vif, 'Tolerance': tolerance})/5

In [27]:
dfMultiCol = df[['operatingMargin','bookToMarketRatio','longFinancialLeverage','EPSP', 'r']].dropna()
calculate_vif(df=dfMultiCol, features=['operatingMargin','bookToMarketRatio','longFinancialLeverage','EPSP'])

,VIF,Tolerance
operatingMargin,0.269281,0.148544
bookToMarketRatio,0.207257,0.192997
longFinancialLeverage,0.227557,0.175780
EPSP,0.252780,0.158240


**Two variables are considered to be perfectly collinear if their correlation coefficient is +/- 1.0. As we can see in the heat map and in the VIF, there are not variable with strong correlation.**

#3 - **Proposes and implements solutions to the problems of the previous points so that the model is the most appropriate.**

**This decisions can be observed when droping the high leverage points and the outliers. Also when the shortFinancialLeverage column was dropped because of the large number of NaN.**

**Therefore the model calculated below is the most accurate possible**

#**4 - Estimate and interpret a Multiple Regression Model after attending the previous problems. You have to use matrix algebra to estimate beta coefficients and standard errors of the regression model. Use Python functions to check results.**

###**Estimate coefficients**

**The betas for a Multiple Linear Regression using matrix algebra can be estimated using the following formula:**
- **(X' X)^−1 X'Y**


In [28]:
#Define independant variables
X = df[['operatingMargin','bookToMarketRatio','longFinancialLeverage','EPSP']]
#Creating a column of ones
N, M = X.shape
ones = np.ones((N,1))
X['ones'] = ones
#Reindex the dataframe so the column of ones appear first
X = X.reindex(columns=['ones','operatingMargin','bookToMarketRatio','longFinancialLeverage','EPSP'])
Xt = X.transpose()
#Define the dependant variable
Y = df['r']
#Using the Linear Algebra formula to estimate the betas 
#b = (X' X)^−1 X'Y
betas = matrix_power(Xt.dot(X), -1).dot(Xt.dot(Y))
betas

array([ 4.47979436, -0.362649  , -1.1488571 , -0.51288365,  9.53498419])

**Now, we will run an OLS model to validate our results**

In [29]:
#Multivariate Linear Regression 
#First we have to drop the Infinite values and Nas
df.replace([np.inf, -np.inf], np.nan, inplace=True)
dfML = df.reset_index().drop(columns=['firm','qdate']).dropna()
#define dependant variable (response)
y = dfML['r']
#define independant variables (predictor)
x = dfML[['operatingMargin','bookToMarketRatio','longFinancialLeverage','EPSP']]
#add constant to predictor variables
x = sm.add_constant(x)
#fit linear regression model
model = sm.OLS(y, x).fit()
#view model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      r   R-squared:                       0.238
Model:                            OLS   Adj. R-squared:                  0.231
Method:                 Least Squares   F-statistic:                     36.45
Date:                Fri, 14 Oct 2022   Prob (F-statistic):           1.59e-26
Time:                        21:04:30   Log-Likelihood:                -644.70
No. Observations:                 472   AIC:                             1299.
Df Residuals:                     467   BIC:                             1320.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     4.47

**As we can see, all the coefitiens are the same, proving that there are more ways to estimate betas when doing linear regression, and one of them is using Matrix Algebra**

**If we would like to estimate prediction we could also use matrix algebra, now that we have the beta values, we could simply use the YH = Xb and we would obtain the predictions for our model. With an accruracy of 23%**

**Now, focusing a little bit on the analysis of the model, we can infer that The t-test assesses whether the beta coefficient is significantly different from zero.  If the beta coefficient is not statistically significant, the variable does not significantly predict the outcome.**

**If the beta coefficient is positive, the interpretation is that for every 1-unit increase in the predictor variable, the outcome variable will increase by the beta coefficient value.  If the beta coefficient is negative, the interpretation is that for every 1-unit increase in the predictor variable, the outcome variable will decrease by the beta coefficient value.**

**The betas we got are:**
- b0 = 4.4798
- b1 = -0.362
- b2 = -1.148
- b3 = -0.512
- b4 = 9.5350

**Which means that if the beta 4 that is EPSP with a coeficient of  9.53 and a statistically significant t of 6.24, then for each 1-unit increase in the predictor variable, the outcome variable will increase by 9.53 units.The beta coefficient is the degree of change in the outcome variable for every 1-unit of change in the predictor variable****

###**Standard errors of the regression model**

**Now that we have te betas, we can calculate the standard error of the beta coeficients, also known as  its standard deviation. This will help us know if each beta coecientis signicantly dierent from zero**

**Then I will calulate the variance of each beta coecient to get their standard deviations, or standard error. Then, let's work with the
variance-covariance matrix of the betas. The variance-covariance matrix can be
expressed as the expected value of the squared dierences between the sample
betas and their respective population betas:**

- **E[(b − ε)(b − β)'] = σ2ε(X'X)^−1**



In [30]:
#Define variables we are going to use, stroring them in the X dataframe
#Define independant variables
X = df[['operatingMargin','bookToMarketRatio','longFinancialLeverage','EPSP']]
#Creating a column of ones
X['ones'] = ones
#Reindex the dataframe so the column of ones appear first
X = X.reindex(columns=['ones','operatingMargin','bookToMarketRatio','longFinancialLeverage','EPSP'])
Xt = X.transpose()
# Obtain how many rows(N) and columns(M) we have
N, M = X.shape
#Defining independant variable
y = df['r']
y = y.to_numpy().reshape(N,1) #Because it has to be of shape nx1
# Create a ones matrix so I can do the (X'1) operations
ones = np.ones((N,1))
XtOnes = Xt.dot(ones)
#Doing the operation that help us obtain the errors matrix:
# E[(b − ε)(b − β)'] = σ2ε(X'X)^−1
#First we have to get the yHat (predictions) to obtsin thre mse
H =  X.dot((matrix_power(Xt.dot(X),-1)).dot(Xt))
yHat = H.dot(y)
#Calculating the σ2ε that is the mean squared error
mse = mean_squared_error(y, yHat)
#Calculating the error matrix
error = mse * (matrix_power(Xt.dot(X), -1))
error



array([[ 0.01377478, -0.00685109, -0.00833989, -0.01562251,  0.0272513 ],
       [-0.00685109,  0.01124504, -0.00018586,  0.01075697, -0.06944446],
       [-0.00833989, -0.00018586,  0.01093681,  0.00306008, -0.02168417],
       [-0.01562251,  0.01075697,  0.00306008,  0.09454855, -0.09233869],
       [ 0.0272513 , -0.06944446, -0.02168417, -0.09233869,  2.30334745]])

**We obtained a matrix, its diagonal will have the variances of the betas. And
the non-diagonal will have the paired covariances of the betas.**

In [31]:
#obtainig the diagonal of the error matrix
error = np.diag(error)
error

array([0.01377478, 0.01124504, 0.01093681, 0.09454855, 2.30334745])

**When calculating the standard errors, each library processes the data with different precisions. Using different precisions can lead to small differences in result but not something that can impact the result or interpretation of the results.**


**The standard error of the operatingMargin, bookToMarketRatio and longFinancialLeverage coefficients are smaller than EPSP. Therefore, this model was able to estimate the coefficient for this 3 variables with greater precision than the EPSP beta. EPSP and bookToMarketRatio have a big t-value whick means that have more statistical significance than the operating margin and longFinancialLeverage.**

**Now, we can reject the null hypothesis, that states that the regression coefficient beta is equal to zero, because by the results obtained we can see that the independent variables are related to the dependent variable.**

#References:

- **Dorantes Dosamantes, C. A. (Aug 27, 2019). Basics of Linear Regression Models in the  context of Finance.**
- **Dorantes Dosamantes, C. A. (Oct 30, 2014). Basics of Portfolio Theory - Part II.**
- **Using Leverages to Help Identify Extreme X Values | STAT 462. Accesed Oct 14, 2022, from https://online.stat.psu.edu/stat462/node/171/**

- **Identifying Outliers (Unusual Y Values) | STAT 462.  Accesed Oct 14, 2022, from  https://online.stat.psu.edu/stat462/node/172/**